# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [2]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# tokenize and stemming

In [3]:
X_data = mbti['posts'].ravel()
y_data = mbti['type'].ravel()

print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [4]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['oi', 'went', 'break', 'month', 'ago', 'togeth', 'year', 'plan', 'life', 'around', 'relationship', 'wasnt', 'one', 'break', 'relationship', 'might', 'imagin', 'ourenfj', 'pun', 'mani', 'punswel', 'person', 'dont', 'go', 'much', 'attract', 'gener', 'see', 'chang', 'that', 'good', 'alreadi', 'may', 'ask', 'want', 'mere', 'sexualsorri', 'infp', 'im', 'realli', 'postrock', 'post', 'go', 'leav', 'sneak', 'thread', 'dthat', 'doesnt', 'sound', 'enfj', 'think', 'x', 'id', 'never', 'act', 'cold', 'toward', 'romant', 'interest', 'get', 'warm', 'side', 'think', 'like', 'someon', 'way', 'want', 'straight', 'themnot', 'realli', 'im', 'mostli', 'guitar', 'player', 'bass', 'obsess', 'im', 'person', 'ask', 'bass', 'louder', 'least', 'relev', 'like', 'tri', 'new', 'instrument', 'sometim', 'add', 'aguitar', 'mostli', 'add', 'bass', 'glockenspiel', 'harmonica', 'drum', 'ocarinaahahah', 'im', 'sorri', 'im', 'laugh', 'hard', 'realli', 'plagu', 'never', 'thought', 'possiblemostli', 

0

# vectorization (tfidf)

In [6]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
y = np.array(y_data)


[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.0518922 0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


# Garbage Collection

In [7]:
import psutil
# import os
# import gc

In [8]:
for clean_sentence in normalized_text:
    del clean_sentence
del normalized_text

del X_data

for component in y_data:
    del component
del y_data

for data in stemData:   # delete 'stemData'
    for component in data:
        del component
    del data
del stemData

gc.collect()

40

# UnderSampling and SMOTE

In [18]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=0).fit_sample(X, y)
X_resampled, y_resampled = RandomUnderSampler(random_state=0,
sampling_strategy={
         'ENFP': 500,
         'ENFJ': 500,
         'ENTJ': 500,
         'ENTP': 500,
         'ESFJ': 500,
         'ESFP': 500,
         'ESTJ': 500,
         'ESTP': 500,
         'INFJ': 500,
         'INFP': 500,
         'INTJ': 500,
         'INTP': 500,
         'ISFJ': 500,
         'ISFP': 500,
         'ISTJ': 500,
         'ISTP': 500
         }).fit_sample(X_resampled, y_resampled)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)

Counter({'ENFJ': 500,
         'ENFP': 500,
         'ENTJ': 500,
         'ENTP': 500,
         'ESFJ': 500,
         'ESFP': 500,
         'ESTJ': 500,
         'ESTP': 500,
         'INFJ': 500,
         'INFP': 500,
         'INTJ': 500,
         'INTP': 500,
         'ISFJ': 500,
         'ISFP': 500,
         'ISTJ': 500,
         'ISTP': 500})

# XGBoost

In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [20]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [22]:
import joblib

joblib.dump(model, 'XGBoost_smoteNunder.pkl')

['XGBoost_smoteNunder.pkl']

In [23]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.6477141759508259
[0.51428571 0.62564103 0.60465116 0.62621359 0.33333333 1.
 0.2        0.58333333 0.64102564 0.66360856 0.62650602 0.6725
 0.6875     0.66101695 0.75675676 0.68292683]
[0.36       0.58937198 0.39393939 0.60280374 0.08333333 0.06666667
 0.125      0.30434783 0.72544643 0.78909091 0.65       0.68101266
 0.41509434 0.45348837 0.42424242 0.62222222]
[0.42352941 0.60696517 0.47706422 0.61428571 0.13333333 0.125
 0.15384615 0.4        0.68062827 0.72093023 0.63803681 0.67672956
 0.51764706 0.53793103 0.54368932 0.65116279]
Evaluation END
